# Clean data

In [7]:
!pip install nltk
!pip install sklearn

In [8]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .getOrCreate()
print(spark)

In [10]:
crypto_df = spark.read.format("mongo").option("uri","mongodb://mongo/seng550.crypto").load()
crypto_df.show()
headers_df = spark.read.format("mongo").option("uri","mongodb://mongo/seng550.headers").load()
headers_df.show()

+--------------------+------------------+----------+------------------+
|                 _id|             close|      date|              open|
+--------------------+------------------+----------+------------------+
|{619f20a05fec62ac...|        61238.6222|2021-10-23| 60690.28731420007|
|{619f20a15fec62ac...| 60694.00000650834|2021-10-22| 62196.96438572152|
|{619f20a15fec62ac...|62298.093007718286|2021-10-21| 65961.04976056097|
|{619f20a15fec62ac...| 65986.30293718613|2021-10-20|        64252.7263|
|{619f20a15fec62ac...|        64217.0839|2021-10-19|        62020.0126|
|{619f20a15fec62ac...|        61953.2331|2021-10-18| 61543.26469764322|
|{619f20a15fec62ac...| 61582.25155249131|2021-10-17| 60867.89483798279|
|{619f20a15fec62ac...| 60860.89183706596|2021-10-16| 61698.03021926293|
|{619f20a15fec62ac...| 61526.33413298988|2021-10-15|57312.383791730215|
|{619f20a15fec62ac...|  57351.7384160736|2021-10-14|        57385.8884|
|{619f20a15fec62ac...|        57357.6067|2021-10-13| 56030.57101

### Clean the crypto dataframe

In [12]:
# https://stackoverflow.com/questions/58823628/call-a-function-for-each-row-of-a-dataframe-in-pysparknon-pandas
from pyspark.sql.functions import col
def get_change(close, open_):
    return  close - open_
spark.udf.register("change_udf", get_change)
# crypto_df.show()
crypto_df = crypto_df.withColumn('change', get_change(col('close'), col('open')))
crypto_df.show()

+--------------------+------------------+----------+------------------+-------------------+
|                 _id|             close|      date|              open|             change|
+--------------------+------------------+----------+------------------+-------------------+
|{619f20a05fec62ac...|        61238.6222|2021-10-23| 60690.28731420007|  548.3348857999299|
|{619f20a15fec62ac...| 60694.00000650834|2021-10-22| 62196.96438572152|-1502.9643792131828|
|{619f20a15fec62ac...|62298.093007718286|2021-10-21| 65961.04976056097| -3662.956752842685|
|{619f20a15fec62ac...| 65986.30293718613|2021-10-20|        64252.7263| 1733.5766371861318|
|{619f20a15fec62ac...|        64217.0839|2021-10-19|        62020.0126|  2197.071299999996|
|{619f20a15fec62ac...|        61953.2331|2021-10-18| 61543.26469764322|  409.9684023567752|
|{619f20a15fec62ac...| 61582.25155249131|2021-10-17| 60867.89483798279|   714.356714508518|
|{619f20a15fec62ac...| 60860.89183706596|2021-10-16| 61698.03021926293| -837.138

21/12/06 19:48:44 WARN SimpleFunctionRegistry: The function change_udf replaced a previously registered function.


### Clean the headers dataframe

In [15]:

import string
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

def clean_header(header):
    porter = PorterStemmer()
    # headers are small, so local process is ok
    header = header.strip()
    if not header:
        return ''
    header = header.translate(str.maketrans('', '', string.punctuation))
    return ' '.join(porter.stem(word) for word in header.split() if word not in ENGLISH_STOP_WORDS)

spark.udf.register("clean_header_udf", clean_header)
headers_df = headers_df.withColumn('clean_header', clean_header(col('headline')))
display(headers_df)

21/12/06 19:49:16 WARN SimpleFunctionRegistry: The function clean_header_udf replaced a previously registered function.


TypeError: 'Column' object is not callable